<a href="https://colab.research.google.com/github/Tessellate-Imaging/Monk_Object_Detection/blob/master/application_model_zoo/Example%20-%20SUIM%20Dataset%20Underwater%20Object%20Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Table of contents


## 1. Installation Instructions



## 2. Use trained model to segment objects in underwater imagery



## 3. How to train a custom segmenter

# About the networks


1. UNet
  - https://arxiv.org/abs/1505.04597
  - https://towardsdatascience.com/understanding-semantic-segmentation-with-unet-6be4f42d4b47
  - https://towardsdatascience.com/unet-line-by-line-explanation-9b191c76baf5


2. FPN
  - http://openaccess.thecvf.com/content_cvpr_2017/papers/Lin_Feature_Pyramid_Networks_CVPR_2017_paper.pdf
  - https://towardsdatascience.com/review-fpn-feature-pyramid-network-object-detection-262fc7482610
  - https://medium.com/@jonathan_hui/understanding-feature-pyramid-networks-for-object-detection-fpn-45b227b9106c


3. PSPNet
  - https://arxiv.org/abs/1612.01105
  - https://towardsdatascience.com/review-pspnet-winner-in-ilsvrc-2016-semantic-segmentation-scene-parsing-e089e5df177d
  - https://developers.arcgis.com/python/guide/how-pspnet-works/


4. Linknet
  - https://arxiv.org/pdf/1707.03718.pdf
  - https://neptune.ai/blog/image-segmentation-tips-and-tricks-from-kaggle-competitions

# Installation

 - Run these commands
     
     - git clone https://github.com/Tessellate-Imaging/Monk_Object_Detection.git
     
     - cd Monk_Object_Detection/9_segmentation_models/installation
     
 - Select the right requirements file and run
 
     - cat requirements_cuda9.0.txt | xargs -n 1 -L 1 pip install

In [ ]:
! git clone https://github.com/Tessellate-Imaging/Monk_Object_Detection.git

In [ ]:
# For colab use the command below
! cd Monk_Object_Detection/9_segmentation_models/installation && cat requirements_colab.txt | xargs -n 1 -L 1 pip install


# For Local systems and cloud select the right CUDA version
#! cd Monk_Object_Detection/9_segmentation_models/installation && cat requirements_cuda10.0.txt | xargs -n 1 -L 1 pip install

# Use already trained model for demo

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/9_segmentation_models/lib/");

In [ ]:
from infer_segmentation import Infer

In [ ]:
gtf = Infer();

In [ ]:
classes_dict = {
    'background': 0, 
    'human_divers': 1,
    'aquatic_pants': 2,
    'wrecks_and_ruins': 3,
    'bots': 4,
    'reefs_and_invertebrates': 5,
    'fish_and_vertebrates': 6,
    'sea_floor': 7
};
classes_to_train = ['background', 'human_divers', 'aquatic_pants', 'wrecks_and_ruins', 'bots', 'reefs_and_invertebrates', 
                    'fish_and_vertebrates', 'sea_floor'];

In [ ]:
gtf.Data_Params(classes_dict, classes_to_train, image_shape=[640, 480])

In [ ]:
# Download trained model

In [ ]:
! wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1G9fk422RbsS-gfj32NJxCLZk9jAlQDjY' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1G9fk422RbsS-gfj32NJxCLZk9jAlQDjY" -O seg_suim_trained.zip && rm -rf /tmp/cookies.txt

In [ ]:
! unzip -qq seg_suim_trained.zip

In [ ]:
gtf.Model_Params(model="Unet", backbone="efficientnetb3", path_to_model='seg_suim_trained/best_model.h5')

In [ ]:
gtf.Setup();

In [ ]:
ls seg_suim_trained/test

In [ ]:
gtf.Predict("seg_suim_trained/test/1.png", vis=True);

In [ ]:
gtf.Predict("seg_suim_trained/test/2.png", vis=True);

In [ ]:
gtf.Predict("seg_suim_trained/test/3.png", vis=True);

In [ ]:
gtf.Predict("seg_suim_trained/test/4.png", vis=True);

In [ ]:
gtf.Predict("seg_suim_trained/test/5.png", vis=True);

In [ ]:
gtf.Predict("seg_suim_trained/test/6.png", vis=True);

In [ ]:
gtf.Predict("seg_suim_trained/test/7.png", vis=True);

In [ ]:
gtf.Predict("seg_suim_trained/test/8.png", vis=True);

# Train you own detector

# Monk Format

## Dataset Directory Structure

      root_dir
          |
          | 
          |         
          |----train_img_dir
          |       |
          |       |---------img1.jpg
          |       |---------img2.jpg
          |                |---------..........(and so on) 
          |
          |----train_mask_dir
          |       |
          |       |---------img1.jpg
          |       |---------img2.jpg
          |                |---------..........(and so on)
          |
          |----val_img_dir (optional)
          |       |
          |       |---------img1.jpg
          |       |---------img2.jpg
          |                |---------..........(and so on)
          |
          |----val_mask_dir (optional)
          |       |
          |       |---------img1.jpg
          |       |---------img2.jpg
          |                |---------..........(and so on)

          
          

# Sample Dataset Credits

    credits: http://irvlab.cs.umn.edu/resources/suim-dataset

In [ ]:
! unzip -qq train_val.zip

In [ ]:
import os
import cv2
import numpy as np 
from tqdm import tqdm

In [ ]:
num = 0;
img_list = os.listdir("train_val/masks");

In [ ]:
for i in tqdm(range(len(img_list))):
    
    img = cv2.imread("train_val/masks/" + img_list[i])
    img = cv2.resize(img, (640, 480))

    img_new = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
    img_new[np.where((img == [0,0,0]))[:2]] = 0
    img_new[np.where((img == [255,0,0]))[:2]] = 1
    img_new[np.where((img == [0,255,0]))[:2]] = 2
    img_new[np.where((img == [255,255,0]))[:2]] = 3
    img_new[np.where((img == [0,0,255]))[:2]] = 4
    img_new[np.where((img == [255,0,255]))[:2]] = 5
    img_new[np.where((img == [0,255,255]))[:2]] = 6
    img_new[np.where((img == [255,255,255]))[:2]] = 7
    #img[img > 6] = [0, 0, 0]

    cv2.imwrite("dataset/masks/" + str(i) + ".png", img_new)


    img = cv2.imread("train_val/images/" + img_list[i].split(".")[0] + ".jpg")
    img = cv2.resize(img, (640, 480))
    cv2.imwrite("dataset/images/" + str(i) + ".png", img)

## Training 

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/9_segmentation_models/lib/");

In [ ]:
from train_segmentation import Segmenter

In [ ]:
gtf = Segmenter();

In [ ]:
train_img_dir = "dataset/images";
train_mask_dir = "dataset/masks";

val_img_dir = "dataset/images";
val_mask_dir = "dataset/masks";


In [ ]:
classes_dict = {
    'background': 0, 
    'human_divers': 1,
    'aquatic_pants': 2,
    'wrecks_and_ruins': 3,
    'bots': 4,
    'reefs_and_invertebrates': 5,
    'fish_and_vertebrates': 6,
    'sea_floor': 7
};
classes_to_train = ['background', 'human_divers', 'aquatic_pants', 'wrecks_and_ruins', 'bots', 'reefs_and_invertebrates', 
                    'fish_and_vertebrates', 'sea_floor'];

In [ ]:
gtf.Train_Dataset(train_img_dir, train_mask_dir, classes_dict, classes_to_train)

In [ ]:
gtf.Val_Dataset(val_img_dir, val_mask_dir)

In [ ]:
gtf.List_Backbones();

In [ ]:
gtf.Data_Params(batch_size=2, backbone="efficientnetb3", image_shape=[640, 480])

In [ ]:
gtf.List_Models();

In [ ]:
gtf.Model_Params(model="Unet")

In [ ]:
gtf.Train_Params(lr=0.0001)

In [ ]:
gtf.Setup();

In [ ]:
gtf.Train(num_epochs=300);

In [ ]:
gtf.Visualize_Training_History();

# Inference

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/9_segmentation_models/lib/");

In [ ]:
from infer_segmentation import Infer

In [ ]:
gtf = Infer();

In [ ]:
classes_dict = {
    'background': 0, 
    'human_divers': 1,
    'aquatic_pants': 2,
    'wrecks_and_ruins': 3,
    'bots': 4,
    'reefs_and_invertebrates': 5,
    'fish_and_vertebrates': 6,
    'sea_floor': 7
};
classes_to_train = ['background', 'human_divers', 'aquatic_pants', 'wrecks_and_ruins', 'bots', 'reefs_and_invertebrates', 
                    'fish_and_vertebrates', 'sea_floor'];

In [ ]:
gtf.Data_Params(classes_dict, classes_to_train, image_shape=[640, 480])

In [ ]:
gtf.Model_Params(model="Unet", backbone="efficientnetb3", path_to_model='best_model.h5')

In [ ]:
gtf.Setup();

In [ ]:
import os
img_list = os.listdir("dataset/images/")

In [ ]:
gtf.Predict("dataset/images/" + img_list[i], vis=True);